In [1]:
# import esm
import torch
from argparse import Namespace
from esm.constants import proteinseq_toks
import math
import torch.nn as nn
import torch.nn.functional as F
from esm.modules import TransformerLayer, PositionalEmbedding  # noqa
from esm.model import ProteinBertModel

# model, alphabet = torch.hub.load("facebookresearch/esm", "esm1_t34_670M_UR50S")
import esm

In [2]:
from ych_util import prepare_mlm_mask
import pandas as pd
import time

In [3]:
dat = pd.read_csv("../../data/kif/kin_kif.csv")
dat = dat.sample(frac = 1)
dat.head()

Entry                                                seq
102  Q9GV92  MVEIMKKSSKQETVKVIVRCRPLSSQEIANNYSKIVHMRPQRGQIE...
89   Q86ZB3  MASSPPESPQVSLHRPVSAMIKGAPRSNSRMSMSSKAGGSRASDED...
18   Q8VWI7  MAPTPSSSRSNQTQYTLIRTPQTKQRLNFHSKTPNPDGSKDPSPPE...
5    Q7Q3T5  GASSSPTKASTPRSVSRPAGSKLATCLLTKRTQLNTSSSESLPASP...
550  Q6YZ52  MATRPASRQRRASSAAAAVAVVRSSPQPQQQQQQQLPIPQSGSPTS...

In [4]:
alphabet = esm.Alphabet.from_dict(proteinseq_toks)
# model_name = "esm1_t34_670M_UR50S"
model_name = "esm1_t12_85M_UR50S"
url = f"https://dl.fbaipublicfiles.com/fair-esm/models/{model_name}.pt"
if torch.cuda.is_available():
    print("cuda")
    model_data = torch.hub.load_state_dict_from_url(url, progress=False)
else:
    model_data = torch.hub.load_state_dict_from_url(url, progress=False, map_location=torch.device('cpu'))

pra = lambda s: ''.join(s.split('decoder_')[1:] if 'decoder' in s else s)
prs = lambda s: ''.join(s.split('decoder.')[1:] if 'decoder' in s else s)
model_args = {pra(arg[0]): arg[1] for arg in vars(model_data["args"]).items()}
model_state = {prs(arg[0]): arg[1] for arg in model_data["model"].items()}

cuda


In [5]:
model = esm.ProteinBertModel(
        Namespace(**model_args), len(alphabet), padding_idx=alphabet.padding_idx
    )

model.load_state_dict(model_state)

<All keys matched successfully>

In [6]:
model.cuda()
model.train()

ProteinBertModel(
  (embed_tokens): Embedding(35, 768, padding_idx=1)
  (embed_positions): PositionalEmbedding()
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (self_attn_layer_norm): BertLayerNorm()
      (fc1): Linear(in_features=768, out_features=3072, bias=True)
      (fc2): Linear(in_features=3072, out_features=768, bias=True)
      (final_layer_norm): BertLayerNorm()
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=Tr

In [7]:
batch_converter = alphabet.get_batch_converter()
criterion = nn.CrossEntropyLoss()
lr = 0.0001 # learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [ ]:
start_time = time.time()
print_every = 300
for j in range(300):
    dat = dat.sample(frac = 1)
    for i in range(dat.shape[0]):
        if len(dat.iloc[i,1])>1024:
            seq = dat.iloc[i,1][:1023]
        else:
            seq = dat.iloc[i,1]
        lab = dat.iloc[i,0]
        data = [(lab, seq)]
        batch_labels, batch_strs, batch_tokens = batch_converter(data)
        true_aa,target_ind,masked_batch_tokens = prepare_mlm_mask(alphabet,batch_tokens)
        optimizer.zero_grad()
        results = model(masked_batch_tokens.to('cuda'), repr_layers=[34])   
        pred = results["logits"].squeeze(0)[target_ind,:]   
        target = true_aa.squeeze(0)
        loss = criterion(pred.cpu(),target)
        loss.backward()
        optimizer.step()

        if i % print_every == 1:
            print(batch_labels)
            print(batch_strs)
            print(batch_tokens.size())
            print(masked_batch_tokens.size())
            print(results["logits"].size())
            print(pred.size())
            print(target.size())
            print(f"At Epoch: %.1f"% i)
            print(f"Loss %.4f"% loss)
            elapsed = time.time() - start_time
            print(f"time elapsed %.4f"% elapsed)
            torch.save(model.state_dict(), "../../data/esm_t12_85M_UR50S_kin_kif_201204.pt")
#         break
#     break

['Q6YZ52']
['MATRPASRQRRASSAAAAVAVVRSSPQPQQQQQQQLPIPQSGSPTSTTTTTTSSSRLTPELSLDGPASPLFAGLDEDPAPKENVTVTVRFRPLSPREIRQGEEVAWYADGDTVVRSEQNPSVAYAYDRVFAPTTTTRQVYDVAAQHVVSGAMEGVNGTIFAYGVTSSGKTHTMHGDQRSPGIIPLAVKDAFSIIQETPNREFLLRVSYLEIYNEVVNDLLNPAGQNLRIREDPQGTFVEGIKEEVVLSPAHALSLIAAGEEHRHVGSTNFNLLSSRSHTIFTLTVESSPCGESNEGEAVTFSQLNLIDLAGSESSRAETTGVRRKEGSYINKSLLTLGTVISKLTDGKATHIPFRDSKLTRLLQSSLSGQGRVSLICTVTPASSNSEETHNTLKFAHRAKRIEVQASQNKIIDEKSLIKKYQNEIRRLKEELEQLKMGIITGTPVKDAGEDNIILWKQKLEDGNVKLQSRLEQEEEAKAALLARIQRLTKLILVSTKATQTSRFSPHPGPRRRHSFGEEELAYLPYKRRDIVLDNESNELLSPVEGLGMTLEDSKEEKKNRKGILNWFKLRKREGGASILTSSEGDKSSLTKSTAPSTPIGESVNFPSEPRISNSLVGESASVDLFSIGHGEFATDSLHGEETPLASRKTIDHVDLLREQLKILSGEVALHTSVLKRLTEEAGRSPNNEKIQMEMKKVNDEIKGKKHQIASLERQIPHSISNNQGMADKLELTPSYAELLEQLNEKSFDLEVKAADNRVIQDQLNEKTTECMELQEEVAHLKEQLYQTLQAKDSLSNSIMMQKNAGINHETDNHADQELSVPREVPGETSPKEPQSVEIDELKQKVCELIEVKAQLETRNQKLLEESTYAKGLASAAGVELKALSEEVTKLMNQNEKLASELASVRSPTPRRANSGLRGTRRDSISRRHEPAPRRDNNAGYEREKALEAVLMEKEQKEAELQRRIEESKQKEAFLESELANMWVLVA

['Q9GQ58']
['MVKGVLKPPQEKNIQVVVRCRPVNSIEKKQNSYSVLDVKPSKKEICVSTEVAEKASSKIFSFDKVFGPKSPQIEVYKSVVAPILDEVLMGYNCTVFAYGQTGTGKTFTMEGERTPDPDLSWEQDPLAGIIPRAMHQIFEKMIGTDIEFSVRVSYLELYNEELFDLLSSQEDTQRLRIFEDSARKGSVVIQGLEEVTVHNKNEVYAILEKGAAKRKTAATLMNAHSSRSHSVFSVTIHIKENSIEGDELLKTGKLNLVDLAGSENIGRSGAVDKRAREAGNINQSLLTLGRVITALVEHAPHVPYRESKLTRLLQDSLGGRTKTSIIATVSPASINVEETLSTLDYAHRAKHITNRPEINQKLTKKALLKEYTEEIEKLRKDLFATREKNGIYLSEEHYKNMEVSIACQRAQIKEMEENIEDLTTQMQKVTELFEFTQKELEERTEELEVTTKNLEETTETLHVTEKDLRVTTQDRDEQCHLVSEHVKTETQLMSEATQLLSTADSSVTDVGGLHAKLDRKRTVEAHNKSAQEVFAESFHSYTSDIKSSLSRLREEQQGKCTSMQQQFEFMISKRTKEAGDLRASLSDMVTSVKGHSAGMMEESQRKREEWEKWSNDACAEHSKFKEDVIDSMNDFHSNRFLTAMNMLAEKLSSLTESLNECRRDIKTQINSQSETVQACIEKQLDQVNVMTTTVEKFSDHQRSKISDLSGQLDMLKDQERHRNQDMMKSIQGLFAQREESFASETNKLQEQLEEASTETERVKTSILSQETSVQESCESFRNVHRESSERLISSADSHHEKTVAQVEDIRQHNTSLEESMTSQTNTQIDRQTQHMEQVCENFKGHMEEISQHVTKQKADKEAMTIEHEASDETLSSTVDKQLTEMTTTVTEWTEDSKARSEDMTSWTAEMEETVESGLKRVEGFLTRDLKEDVPTGTTPQRKHFSYPRDLTSTEPHGIILNRFRQECELNQAADMPLPDDSIASECP

['Q96L93']
['MASVKVAVRVRPMNRREKDLEAKFIIQMEKSKTTITNLKIPEGGTGDSGRERTKTFTYDFSFYSADTKSPDYVSQEMVFKTLGTDVVKSAFEGYNACVFAYGQTGSGKSYTMMGNSGDSGLIPRICEGLFSRINETTRWDEASFRTEVSYLEIYNERVRDLLRRKSSKTFNLRVREHPKEGPYVEDLSKHLVQNYGDVEELMDAGNINRTTAATGMNDVSSRSHAIFTIKFTQAKFDSEMPCETVSKIHLVDLAGSERADATGATGVRLKEGGNINKSLVTLGNVISALADLSQDAANTLAKKKQVFVPYRDSVLTWLLKDSLGGNSKTIMIATISPADVNYGETLSTLRYANRAKNIINKPTINEDANVKLIRELRAEIARLKTLLAQGNQIALLDSPTALSMEEKLQQNEARVQELTKEWTNKWNETQNILKEQTLALRKEGIGVVLDSELPHLIGIDDDLLSTGIILYHLKEGQTYVGRDDASTEQDIVLHGLDLESEHCIFENIGGTVTLIPLSGSQCSVNGVQIVEATHLNQGAVILLGRTNMFRFNHPKEAAKLREKRKSGLLSSFSLSMTDLSKSRENLSAVMLYNPGLEFERQQREELEKLESKRKLIEEMEEKQKSDKAELERMQQEVETQRKETEIVQLQIRKQEESLKRRSFHIENKLKDLLAEKEKFEEERLREQQEIELQKKRQEEETFLRVQEELQRLKELNNNEKAEKFQIFQELDQLQKEKDEQYAKLELEKKRLEEQEKEQVMLVAHLEEQLREKQEMIQLLRRGEVQWVEEEKRDLEGIRESLLRVKEARAGGDEDGEELEKAQLRFFEFKRRQLVKLVNLEKDLVQQKDILKKEVQEEQEILECLKCEHDKESRLLEKHDESVTDVTEVPQDFEKIKPVEYRLQYKERQLQYLLQNHLPTLLEEKQRAFEILDRGPLSLDNTLYQVEKEMEEKEEQLAQYQANANQLQKLQATFEFTANIARQEEKVR

['A0A3P6AB31']
['MDAMRKQLDVLMGANRNGDVTEVNRKYYDRDVCRLYLSGLCAHDLFQLTKMDMSPCPKVCTLCSSGKNIKKQRQKVLEDAIDKAQGTHLVALDRHEPFKTQVECDRKIGRALKRLQEEDAKAAIAISVTQVTQSPEIQEADLHGNLQKKKKKNLICLIDHQWFLSPSFFEVLEEMRTKRADLQAVLLLEAFNKDRAALPTPAQQPSAALPPPNPRTQEMINEKLKKAEEFGELGMVDEAQKALEEAEALKKLTPRQEPVVDPTKYTAADVRITDQKLRLCDICGAFLSIYDSDRRLADHFGGKLHLGYMLIRDKLAELQEEKNKVNKERVEERRSKERSRERESSKERDRGDSRDRETIISHAGMTQEAGAVRVPGRGKDQEIMIAADAMTAIKTAGDSCSKFKKFFEIRLESI']
torch.Size([1, 415])
torch.Size([1, 415])
torch.Size([1, 415, 35])
torch.Size([62, 35])
torch.Size([62])
At Epoch: 301.0
Loss 2.5969
time elapsed 2985.4071
['Q9Y496']
['MPINKSEKPESCDNVKVVVRCRPLNEREKSMCYKQAVSVDEMRGTITVHKTDSSNEPPKTFTFDTVFGPESKQLDVYNLTARPIIDSVLEGYNGTIFAYGQTGTGKTFTMEGVRAIPELRGIIPNSFAHIFGHIAKAEGDTRFLVRVSYLEIYNEEVRDLLGKDQTQRLEVKERPDVGVYIKDLSAYVVNNADDMDRIMTLGHKNRSVGATNMNEHSSRSHAIFTITIECSEKGIDGNMHVRMGKLHLVDLAGSERQAKTGATGQRLKEATKINLSLSTLGNVISALVDGKSTHVPYRNSKLTRLLQDSLGGNSKTMMCANIGPADYNYDETISTLRYANRAKNIKNKARINEDPKDALLRQFQKEIEELKKKLEEGEEISGSDISGSEEDDDEEGE

['P20480']
['MESRLPKPSGLKKPQMPIKTVLPTDRIRAGLGGGAAGAGAFNVNANQTYCGNLLPPLSRDLNNLPQVLERRGGGARAASPEPMKLGHRAKLRRSRSACDINELRGNKRTAAAPSLPSIPSKVSRLGGALTVSSQRLVRPAAPSSITATAVKRPPVTRPAPRAAGGAAAKKPAGTGAAASSGAAAAAPKRIAPYDFKARFHDLLEKHKVLKTKYEKQTEDMGELESMPQQLEETQNKLIETESSLKNTQSDNECLQRQVKQHTAKIETITSTLGRTKEELSELQAIHEKVKTEHAALSTEVVHLRQRTEELLRCNEQQAAELETCKEQLFQSNMERKELHNTVMDLRGNIRVFCRIRPPLESEENRMCCTWTYHDESTVELQSIDAQAKSKMGQQIFSFDQVFHPLSSQSDIFEMVSPLIQSALDGYNICIFAYGQTGSGKTYTMDGVPESVGVIPRTVDLLFDSIRGYRNLGWEYEIKATFLEIYNEVLYDLLSNEQKDMEIRMAKNNKNDIYVSNITEETVLDPNHLRHLMHTAKMNRATASTAGNERSSRSHAVTKLELIGRHAEKQEISVGSINLVDLAGSESPKTSTRMTETKNINRSLSELTNVILALLQKQDHIPYRNSKLTHLLMPSLGGNSKTLMFINVSPFQDCFQESVKSLRFAASVNSCKMTKAKRNRYLNNSVANSSTQSNNSGSFDK']
torch.Size([1, 701])
torch.Size([1, 701])
torch.Size([1, 701, 35])
torch.Size([105, 35])
torch.Size([105])
At Epoch: 301.0
Loss 1.5918
time elapsed 4250.5741
['A0A3P6AVB3']
['IYLPNPSHHYGSLAREAFLVSELIVPIDCLGIEKKDCKKIGAKLRDMYGPLIYGDIAKSRS']
torch.Size([1, 62])
torch.Size([1, 62])
torch

...

['Q7X7H4']
['MGAIGGDEVVQWDKMDGGEVVNGGGGGGVGKLERILVSVRLRPLSDKEIARGDPSEWECINDTTIISRSTFPDRPSAPTAYSFDRVFRSDCDTNEVYKQGAKEVALSVVSGINSSIFAYGQTSSGKTYTMTGITEYTVADIYDYIGKHEERAFVLKFSAIEIYNEVVRDLLSAENTPLRLWDDAEKGTYVENLTEVVLRDWNHLKELISVCEAQRKTGETYLNENSSRSHQILKLTIESSAREFLGKDKSTTLVASVNFVDLAGSERASQALSAGARLKEGCHINRSLLTLGTVIRKLSKVRNGHIPYRDSKLTRILQPSLGGNARTAIICTMSPARSHMEQSRNTLLFASCAKEVVTNAQVNVVMSDKALVKQLQKELARLESELRCPASYSSLESLVKEKDNQIRKMEKEIKELKLQRDLAQSRLQDLLQVVGDNHVHVSKQSSVSGRNFTFDVPQTCEDEQSTTESSEVVDSVQNFRFQGRRVAQREHKPQQAENNVQFTTPSRYSVSSPPFSGMLPTNRSDHLSQISNEDSDDICKEVRCIETNETGGNECLESSAVGSNSLQDPNAGSSMHINNDSNSSMNSRLRDESPVTLEQHLENVRKPFANIVKDLGSSTRNSSSSKVLGRSRSCRSLTGSSLFEDLEKDDCTPPNRSFIDFAGRPQNCQRRGSALNYDAESETLSRAGSMLSEITTTRDGLKANSSVAGDTEFTGIGEFVAELKEMAQVQYQKQLGHSGNGDLAEGTIRSVGLDPITDALQSPSRWPLEFEKKQQEIIDFWHACNVSLVHRTYFFLLFKGDPADSIYMEVELRRLSFLKDTYSNGAIASIPNTSLVSSAKKLQREREMLCRQMQRRLSIEERESMYTKWGVSLASKRRRLQVARCLWTETKDLEHVRESASLVARLIGLLEPGKALREMFGLSFAPQQFTRRSYNSWRYGRSSLN']
torch.Size([1, 946])
torch.Size([1, 946

['P97329']
['MSHRILSPPAGLLSDEDVVDSPILESTAADLRSVVRKDLLSDCSVISASLEDKQALLEDTSEKVKVYLRIRPFLTSELDRQEDQGCVCIENTETLVLQAPKDSFALKSNERGVGQATHKFTFSQIFGPEVGQVAFFNLTMKEMVKDVLKGQNWLIYTYGVTNSGKTYTIQGTSKDAGILPQSLALIFNSLQGQLHPTPDLKPLLSNEVIWLDSKQIRQEEMKKLSLLIGGLQEEELSTSVKKRVHTESRIGASNSFDSGVAGLSSTSQFTSSSQLDETSQLWAQPDTVPVSVPADIRFSVWISFFEIYNELLYDLLEPPSHQHKRQTLRLCEDQNGNPYVKDLNWIHVRDVEEAWKLLKVGRKNQSFASTHMNQQSSRSHSIFSIRILHLQGEGDIVPKISELSLCDLAGSERCKHQKSGERLKEAGNINTSLHTLGRCIAALRQNQQNRSKQNLIPFRDSKLTRVFQGFFTGRGRSCMIVNVNPCASTYDETLHAAKFSALASQLVHAPPVHLGIPSLHSFIKKHSPQVGPGLEKEDKADSDLEDSPEDEADVSVYGKEELLQVVEAMKALLLKERQEKLQLEIQLREEICNEMVEQMQQREQWCSERLDNQKELMEELYEEKLKILKESLTTFYQEQIQERDEKIEELETLLQEAKQQPAAQQSGGLSLLRRSQRLAASASTQQFQEVKAELEQCKTELSSTTAELHKYQQVLKPPPPAKPFTIDVDKKLEEGQKNIRLLRTELQKLGQSLQSAERACCHSTGAGKLRQALTNCDDILIKQNQTLAELQNNMVLVKLDLQKKAACIAEQYHTVLKLQGQASAKKRLGANQENQQPNHQPPGKKPFLRNLLPRTPTCQSSTDSSPYARILRSRHSPLLKSPFGKKY']
torch.Size([1, 888])
torch.Size([1, 888])
torch.Size([1, 888, 35])
torch.Size([133, 35])
torch.Si

['Q86ZA9']
['VRRRLRNEVEDLESRHRKDTDRLERERDEVERKTRMEFEARIDKLLKEHEEEIQVLRKKLDSDSEAERSRRAQEAQKIEEEYASKLRAAALDADVKQREAQLLQGELTNVKSELDRERTLKTGLQNQLTEATTHNLTLEAANKAMKEKINFLESDSQAQSSAFNDLHKRMQDAIEAAERAHDKLRQEETLRRKLFNQVQELKGNIRVMCRVRPAHDTERDPAQISFPDNDTDSKEVAITGPSKQSATGKDITASYSYSFDRVFGPASQNGEVFEEISQLVQSALDGYNVCIFCYGQTGSGKTHTMSSADGMIPRATAQIWDEAQRLQEKGWRYTMEGSFIEVYNETYNDLLGRSEDLDKKKVEVRHDPVKKQTNLDNAVSVMLDGPGRVEEILETASKNRTVAATKANMRSSRSHSVFILRLVGTNDITGERSEGTLNLVDLAGSERLEHSKVEGARLKETQNINKSLSCLGDVINALGSAKEGGHIPYRNSKLTYLLQYSLGGNSKTLMFVMVSPLQAHLQETITSLKFATKVHNTHIGTAKKQTKTSSTMGWSNGEIFRPLLGWVLGFLGWHDNVWKGKGGFYTHIFPCLLGFAG']
torch.Size([1, 598])
torch.Size([1, 598])
torch.Size([1, 598, 35])
torch.Size([89, 35])
torch.Size([89])
At Epoch: 1.0
Loss 0.0871
time elapsed 69932.5470
['Q9BPU3']
['MEKRQLYSSQSQSQPLNIITNTINSRPSLLRKPASSSSQSNDRISYPPSTDSKFIQQQYHQPLLTNTDIKLEDIESSSSNNNPLKNSINNVSMQISQLNSSHHSRALLMQKRNNPTTNIRPTVKKKLDDTHKPLTSNFKKPITPISKLNTNMNNNNINNKNNNININSNNSSNSNNNILSPVQNNTISPNSNLLNSSIKFEKSNFFSTMYSSPTTITTT

['B9FMJ3']
['MGDSGDAVMARWLQSAGLQHLAASSTSSSSASTAGGGVDPRGGGGVGVGALGGGAGGGSLLPSLLMQGYGPQSIEEKQRLYMLLRSLNFNGETAPPSISEPYTPTAQSFGGGNSLEGFYSPELRGELGAGLLDLHAMDDTELLSEDVASEPFEPSPFIPKEMDEDDDDMLPGSQPGPSDNYNAVANEKESTARENNVAKIKVVVRKRPLNRKEVSRKEEDIITVHDSSSLTVYEPKLKVDLTAYVEKHEFCFDAVLDEQVSNDEVYRETVEPIIPIIFQRTKATCFAYGQTGSGKTYTMQPLPLRAAQDMVRLLHQPVYRNQNFKLWLSYFEIYGGKLFDLLSDRRQLLMREDGKKQVCIVGLQEFEVSDVQIVKEYIERGNAARSTGSTGANEESSRSHAILQLAIKKHIIVTDTRRQRDRDANESKNTKAVGKISFIDLAGSERGADTTDNDRQTRIEGAEINKSLLALKECIRALDNDQIHIPFRGSKLTEVLRDSFVGNSRTVMISCISPNAGSCEHTLNTLRYADRVKSLSKGSNTRKEQPTGPTIPSSKDSSSAPSYPMPIETEEIANQIQEKRPVETSRKAAENFTSNSSMEPDRNPVSMIPSYSNRGKEENGSSGLNDRERVDLNSSRISYNSKPQSVQSSANLQEEEKVTKVSPPRRKAYRDDKPERQSNYAKKDSGPETSRPGYKVQQAKQLQQQQRPTSASASQNSSRQSEKESSCDDVEIDAILEEEEALIAAHRKEIENTMEIVREEMNLLAEVDQPGSLIDNYVTQLSFLLSRKAAGLVSLQARLARFQHRLKEQEILSRKKSSR']
torch.Size([1, 820])
torch.Size([1, 820])
torch.Size([1, 820, 35])
torch.Size([122, 35])
torch.Size([122])
At Epoch: 1.0
Loss 0.1170
time elapsed 70931.5137
['Q0JLE4

['Q9LDN0']
['MKKHFTLPRNAILRDGGEPHSPNPSISKSKPPRKLRSAKENAPPLDRNTSTPDHRSMRMKNPLPPRPPPSNPLKRKLSAETATESGFSDSGVKVIVRMKPLNKGEEGDMIVEKMSKDSLTVSGQTFTFDSIANPESTQEQMFQLVGAPLVENCLSGFNSSVFAYGQTGSGKTYTMWGPANGLLEEHLCGDQRGLTPRVFERLFARIKEEQVKHAERQLNYQCRCSLLEIYNEQITDLLDPSQKNLMIREDVKSGVYVENLTEEYVKNLTDVSQLLIKGLGNRRTGATSVNTESSRSHCVFTCVVESRCKNVADGLSSFKTSRINLVDLAGSERQKSTGAAGERLKEAGNINRSLSQLGNLINILAEISQTGKPRHIPYRDSRLTFLLQESLGGNAKLAMVCAVSPSQSCRSETFSTLRFAQRAKAIQNKAVVNEVMQDDVNFLRGVIHQLRDELQRMKNDGNNPTNPNVAYSTAWNARRSLNLLRSFGLGHPRSLPHEDNDGDIEMEIDEAAVERLCVQVGLQSSLASEGINHDMNRVKSIHSSDGQSIEKRLPEDSDVAMEDACCHTENHEPETVDNMRTETETGIRENQIKTHSQTLDHESSFQPLSVKDALCSSLNKSEDVSSCPDLVPQDVTSANVLIADGVDDPEHLVNSASPSLCIDPVGATPVLKSPTLSVSPTIRNSRKSLKTSELSTASQKDSEGENLVTEAADPSPATSKKMNNCSSALSTQKSKVFPVRTERLASSLHKGIKLLESYCQSTAQRRSTYRFSFKAPDSEPSTSISKADAGVQTIPGADAISEENTKEFLCCKCKCREQFDAQQMGDMPNLQLVPVDNSEVAEKSKNQVPKAVEKVLAGSIRREMALEEFCTKQASEITQLNRLVQQYKHERECNAIIGQTREDKIIRLESLMDGVLSKEDFLDEEFASLLHEHKLLKDMYQNHPEVLKTKIELERTQEEVENFKNFYGDMGEREVLLEEIQDLKLQL

In [11]:
torch.save(model.state_dict(), "../../data/esm_t12_85M_UR50S_kin_kif_201204.pt")

In [12]:
print("done")

done
